In [320]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader as loader

In [279]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cv1=nn.Conv2d(3,32,kernel_size=3,stride=1, padding=1)
        self.cv2=nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
        self.pool=nn.MaxPool2d(2,2)
        self.fc1=nn.Linear(64*8*8,128)
        self.fc2=nn.Linear(128,10)

    def forward(self,x):
        x=F.relu(self.cv1(x))
        x=self.pool(x)
        x=F.relu(self.cv2(x))
        x=self.pool(x)
        x=x.view(-1,64*8*8)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x



In [322]:
from torchvision import models
import torch.nn as nn

# 사전 학습된 ResNet18 불러오기
model = models.resnet18(pretrained=True)
# 마지막 Fully Connected 레이어 수정 (ImageNet의 1000 클래스 → CIFAR-10의 10개 클래스)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)


In [324]:
def train_mode(model, train, optim, criter, num_epoch):
    model.train()
    total_L=0.0
    ave_loss=0.0
    for epoch in range(num_epoch):
        for data,target in train:
            optim.zero_grad()
            out=model(data)
            loss=criter(out,target)
            optim.step()
            total_L+=loss.item()
        avg_loss=loss/len(train)
        print(f"Epoch: {epoch+1}/{num_epoch}, Train Loss: {avg_loss:.4f}")
    return avg_loss



In [326]:
import optuna

def objective(trial):
    lr=trial.suggest_loguniform('lr', 1e-4, 1e-2)
    bs=trial.suggest_categorical('batch_size', [32, 64, 128])

    trin_loader=loader(train_data,batch_size=bs,shuffle=True)
    test_loader=loader(test_data,batch_size=bs,shuffle=True)

    
    model=simple_CNN()
    criter=nn.CrossEntropyLoss()
    optimy=optim.Adam(model.parameters(),lr=lr)

    train_mode(model,train_loader, criter, optimz, num_epoch)
    test_acc=eval_mode(model, test_loader)
    return test_acc

In [328]:
def eval_mode(model,test_loader):
    model.eval()
    correct=0
    total=0
    with torch.no_grad():
        for data,target in test_loader:
            out=model(data)
            pred=out.argmax(dim=1)
            correct+=pred.eq(target).sum().item()
            total+=data.size(0)

    accuracy=100.0*correct/total
    return accuracy

In [330]:
from torchvision import transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


train_data=datasets.CIFAR10(root="./data",train=True, download=True, transform=transform)
test_data=datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader=loader(train_data, batch_size=64, shuffle=True)
test_loader=loader(test_data, batch_size=1000, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [332]:
print(train_data)
print("////////////////////////////////")
print(test_data)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )
////////////////////////////////
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )


In [334]:
model=Model()
criter=nn.CrossEntropyLoss()
optimy=optim.Adam(model.parameters(),lr=0.001)
num_epoch=10

train_mode(model, train_loader, optimy, criter, num_epoch)

In [336]:
#하이퍼파라미터 튜닝
import itertools

lrs = [0.001, 0.005, 0.0001]
batch_sizes = [32, 64, 128]
result = {}

for lr_val, bs in itertools.product(lrs, batch_sizes):
    print(f"Train LR = {lr_val}, batch_size = {bs}")
    
    # 모델 재초기화: 각 실험마다 모델을 새롭게 초기화하는 것이 좋습니다.
    model = Model()  # 혹은 simple_CNN() 등 초기 모델을 재생성합니다.

    # DataLoader도 새롭게 구성합니다.
    train_loader = loader(train_data, batch_size=bs, shuffle=True)
    test_loader = loader(test_data, batch_size=1000, shuffle=True)
    
    # 옵티마이저 생성 시, 현재 lr_val 사용
    optimizer = optim.Adam(model.parameters(), lr=lr_val)
    
    # 학습 함수 호출: 함수의 인자 이름과 전달하는 변수 이름이 일치해야 함
    train_mode(model, train_loader, optimy, criter, num_epoch)
    
    acc = eval_mode(model, test_loader)
    result[(lr_val, bs)] = acc
    print(f"ACCURACY: {acc:.2f}%\n")



Train LR = 0.001, batch_size = 32
Epoch: 1/10, Train Loss: 0.0015
Epoch: 2/10, Train Loss: 0.0015
Epoch: 3/10, Train Loss: 0.0015
Epoch: 4/10, Train Loss: 0.0015
Epoch: 5/10, Train Loss: 0.0015
Epoch: 6/10, Train Loss: 0.0015
Epoch: 7/10, Train Loss: 0.0015
Epoch: 8/10, Train Loss: 0.0015
Epoch: 9/10, Train Loss: 0.0015
Epoch: 10/10, Train Loss: 0.0015
ACCURACY: 10.04%

Train LR = 0.001, batch_size = 64
Epoch: 1/10, Train Loss: 0.0030
Epoch: 2/10, Train Loss: 0.0029
Epoch: 3/10, Train Loss: 0.0030
Epoch: 4/10, Train Loss: 0.0029
Epoch: 5/10, Train Loss: 0.0029
Epoch: 6/10, Train Loss: 0.0029
Epoch: 7/10, Train Loss: 0.0030
Epoch: 8/10, Train Loss: 0.0029
Epoch: 9/10, Train Loss: 0.0029
Epoch: 10/10, Train Loss: 0.0029
ACCURACY: 9.90%

Train LR = 0.001, batch_size = 128
Epoch: 1/10, Train Loss: 0.0059
Epoch: 2/10, Train Loss: 0.0059
Epoch: 3/10, Train Loss: 0.0059
Epoch: 4/10, Train Loss: 0.0059
Epoch: 5/10, Train Loss: 0.0059
Epoch: 6/10, Train Loss: 0.0059
Epoch: 7/10, Train Loss: 0.0

In [318]:
#train_mode(model, train_loader, criter, optimy, num_epoch)
eval=eval_mode(model, test_loader)
print(f"test acc: {eval:.4f}%")

test acc: 9.2100%


In [180]:
def eval_mode(model, test):
    model.eval()
    total=0
    correct=0
    with torch.no_grad():
        for data,target in test:
            out=model(data)
            pred=out.argmax(dim=0)
            correct+=pred.eq(target).sum().item()
            total+=data.size(0)
    accuracy=100.0*correct/total
    return accuracy



In [338]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        # 첫 번째 Convolution + BatchNorm
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        # 두 번째 Convolution + BatchNorm
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # 세 번째 Convolution + BatchNorm 추가
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # MaxPooling 레이어
        self.pool = nn.MaxPool2d(2, 2)
        
        # Global Average Pooling: 입력 feature map을 각 채널의 평균으로 만듭니다.
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        
        # 출력층: GAP 후의 feature의 차원은 128
        self.fc = nn.Linear(128, 10)
        
        # Dropout (과적합 방지)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # x: (batch_size, 3, 32, 32) CIFAR-10 이미지
        x = F.relu(self.bn1(self.conv1(x)))  # → (batch_size, 32, 32, 32)
        x = self.pool(x)                     # → (batch_size, 32, 16, 16)
        
        x = F.relu(self.bn2(self.conv2(x)))  # → (batch_size, 64, 16, 16)
        x = self.pool(x)                     # → (batch_size, 64, 8, 8)
        
        x = F.relu(self.bn3(self.conv3(x)))  # → (batch_size, 128, 8, 8)
        # Global Average Pooling: 각 채널 별로 8x8을 평균 → (batch_size, 128, 1, 1)
        x = self.gap(x)
        
        x = x.view(x.size(0), -1)            # flatten → (batch_size, 128)
        x = self.dropout(x)
        x = self.fc(x)                       # → (batch_size, 10)
        return x

# 모델 인스턴스 생성
model_improved = ImprovedCNN()
print(model_improved)


ImprovedCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (gap): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [340]:
criter=nn.CrossEntropyLoss()
optimy=optim.Adam(model.parameters(),lr=0.001)
num_epoch=10

train_mode(model, train_loader, optimy, criter, num_epoch)

Epoch: 1/10, Train Loss: 0.0059
Epoch: 2/10, Train Loss: 0.0059
Epoch: 3/10, Train Loss: 0.0059
Epoch: 4/10, Train Loss: 0.0059
Epoch: 5/10, Train Loss: 0.0059
Epoch: 6/10, Train Loss: 0.0059
Epoch: 7/10, Train Loss: 0.0059
Epoch: 8/10, Train Loss: 0.0059
Epoch: 9/10, Train Loss: 0.0059
Epoch: 10/10, Train Loss: 0.0059


tensor(0.0059, grad_fn=<DivBackward0>)

In [343]:
eval=eval_mode(model, test_loader)
print(f"test acc: {eval:.4f}%")

test acc: 9.0800%


In [ ]:
모델 최적화 및 해석 (Explainable AI), 생성 모델 (Generative Models), 강화학습 (Reinforcement Learning), 딥러닝 배포 및 서비스화